In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV


In [2]:
iris = load_iris()
X, y = iris.data, iris.target

# Normalize the features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [3]:
class ELMNetwork:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.weights = np.random.uniform(-1, 1, (self.input_dim, self.hidden_dim))
        self.biases = np.random.uniform(-1, 1, (self.hidden_dim,))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def train(self, X, y):
        H = self.sigmoid(np.dot(X, self.weights) + self.biases)
        H_inv = np.linalg.pinv(H)
        self.output_weights = np.dot(H_inv, y)

    def predict(self, X):
        H = self.sigmoid(np.dot(X, self.weights) + self.biases)
        predictions = np.dot(H, self.output_weights)
        return np.argmax(predictions, axis=1)


In [4]:
param_grid = {'hidden_dim': np.arange(10, 101)}


In [24]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin

class ELMNetwork(BaseEstimator, ClassifierMixin):
    def __init__(self, hidden_dim):
        self.hidden_dim = hidden_dim
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        self.input_dim = X.shape[1]
        self.output_dim = len(np.unique(y))
        self.weights = np.random.uniform(-1, 1, (self.input_dim, self.hidden_dim))
        self.biases = np.random.uniform(-1, 1, (self.hidden_dim,))

        H = self.sigmoid(np.dot(X, self.weights) + self.biases)
        H_inv = np.linalg.pinv(H)
        self.output_weights = np.dot(H_inv, y)

    def predict(self, X):
        H = self.sigmoid(np.dot(X, self.weights) + self.biases)
        predictions = np.dot(H, self.output_weights)
        return np.argmax(predictions, axis=1).reshape(-1, 1)


# Load and preprocess the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Normalize the features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the parameter grid for RandomizedSearchCV
param_grid = {'hidden_dim': np.arange(10, 101)}

# Create an instance of the ELMNetwork class
elm = ELMNetwork(hidden_dim=10)

# Create RandomizedSearchCV instance
random_search = RandomizedSearchCV(estimator=elm, param_distributions=param_grid, n_iter=10, scoring='accuracy')

# Fit the ELM network and find the optimal number of hidden nodes
random_search.fit(X_train, y_train)

# Get the best parameters
best_hidden_dim = random_search.best_params_['hidden_dim']
print("Best number of hidden nodes:", best_hidden_dim)

# Create a new instance of the ELMNetwork class with the optimal number of hidden nodes
elm = ELMNetwork(hidden_dim=best_hidden_dim)

# Fit the ELM network with the optimal number of hidden nodes
elm.fit(X_train, y_train)

# Make predictions on the test set
y_pred = elm.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Best number of hidden nodes: 98


C:\Users\salma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\salma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\salma\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\salma\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\salma\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "C:\Users\salma\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 84, in _check_targets
    check_

TypeError: Singleton array 8 cannot be considered a valid collection.

In [25]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

iris = load_iris()
x, y = iris.data, iris.target

scale = StandardScaler()
scaled_data = scale.fit_transform(x)

ohe = OneHotEncoder()
ohe_data = ohe.fit_transform(y.reshape(-1, 1)).toarray()

x_train, x_test, y_train, y_test = train_test_split(scaled_data, ohe_data, test_size=0.3)

def create_model(learning_rate=0.001, activation='sigmoid', hidden_layer_sizes=(50, 20)):
    model = Sequential()
    model.add(Dense(units=hidden_layer_sizes[0], activation=activation))
    model.add(Dense(units=hidden_layer_sizes[1], activation=activation))
    model.add(Dense(units=3, activation='softmax'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model)

param = {
    "learning_rate": [0.01, 0.001, 0.0001],
    "activation": ["sigmoid", "relu", "softmax"],
    "hidden_layer_sizes": [(50, 20), (30, 30), (40, 10)]
}

Random_search = RandomizedSearchCV(model, param, cv=5, n_iter=5, random_state=10)
Random_search.fit(x_train, y_train)


C:\Users\salma\AppData\Local\Temp\ipykernel_36256\75516265.py:32: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model)


4/4 [==============================] - 1s 5ms/step - loss: 0.6844 - accuracy: 0.3429


RandomizedSearchCV(cv=5,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001D0966EFA00>,
                   n_iter=5,
                   param_distributions={'activation': ['sigmoid', 'relu',
                                                       'softmax'],
                                        'hidden_layer_sizes': [(50, 20),
                                                               (30, 30),
                                                               (40, 10)],
                                        'learning_rate': [0.01, 0.001, 0.0001]},
                   random_state=10)